In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Load MovieLens dataset (UPDATE THE FILE PATH BELOW)
file_path = "/content/ratings.csv"  # <-- Change this!
df = pd.read_csv(file_path)

# Sort data by user and timestamp
df = df.sort_values(by=['userId', 'timestamp'])

# Create sequences of movie interactions for each user
user_sequences = df.groupby('userId')['movieId'].apply(list)

# Convert movie IDs to a contiguous range
unique_movies = list(set(df['movieId']))
movie_to_index = {movie: i + 1 for i, movie in enumerate(unique_movies)}  # Reserve 0 for padding
index_to_movie = {i + 1: movie for i, movie in enumerate(unique_movies)}
df['movieId'] = df['movieId'].map(movie_to_index)

# Prepare input-output sequences (X: history, y: next movie)
sequence_length = 5
X, y = [], []

for user_movies in user_sequences:
    indexed_movies = [movie_to_index[m] for m in user_movies if m in movie_to_index]
    for i in range(len(indexed_movies) - sequence_length):
        X.append(indexed_movies[i:i + sequence_length])
        y.append(indexed_movies[i + sequence_length])

X = pad_sequences(X, maxlen=sequence_length, padding='pre')
y = np.array(y)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
vocab_size = len(movie_to_index) + 1  # +1 for padding index
embedding_dim = 50

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length),
    LSTM(64, return_sequences=False),
    Dense(vocab_size, activation='softmax')  # Predicting next movie ID
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model (reduced epochs for quick testing)
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Print model summary
model.summary()

# Function to predict next movie for a given sequence
def recommend_next_movie(movie_sequence):
    movie_sequence = [movie_to_index.get(m, 0) for m in movie_sequence]  # Convert to index
    movie_sequence = pad_sequences([movie_sequence], maxlen=sequence_length, padding='pre')
    predicted_index = np.argmax(model.predict(movie_sequence), axis=-1)[0]
    return index_to_movie.get(predicted_index, "Unknown Movie")

# Example usage: Predict next movie for a user sequence
example_sequence = list(user_sequences.iloc[0])[:sequence_length]  # Get first user sequence
predicted_movie = recommend_next_movie(example_sequence)
print("Predicted next movie ID:", predicted_movie)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
11877/11877 ━━━━━━━━━━━━━━━━━━━━ 109s 9ms/step - accuracy: 0.0030 - loss: 8.2477 - val_accuracy: 0.0076 - val_loss: 7.5790
Epoch 2/5
11877/11877 ━━━━━━━━━━━━━━━━━━━━ 105s 9ms/step - accuracy: 0.0090 - loss: 7.3715 - val_accuracy: 0.0125 - val_loss: 7.4243
Epoch 3/5
11877/11877 ━━━━━━━━━━━━━━━━━━━━ 149s 9ms/step - accuracy: 0.0145 - loss: 7.0967 - val_accuracy: 0.0155 - val_loss: 7.3826
Epoch 4/5
11877/11877 ━━━━━━━━━━━━━━━━━━━━ 104s 9ms/step - accuracy: 0.0203 - loss: 6.8786 - val_accuracy: 0.0183 - val_loss: 7.3778
Epoch 5/5
11877/11877 ━━━━━━━━━━━━━━━━━━━━ 150s 9ms/step - accuracy: 0.0258 - loss: 6.7063 - val_accuracy: 0.0201 - val_loss: 7.4009


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 5, 50)               │       1,057,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          29,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 21156)               │       1,375,140 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,387,142 (28.18 MB)

 Trainable params: 2,462,380 (9.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,924,762 (18.79 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
Predicted next movie ID: 3160


# New section